### Carga de las imagenes

In [1]:
from image_services import get_image_urls
from skimage import io
import solar_image_analysis as sia
import re 
from datetime import datetime

Acá esta como ejemplo como podemos generar todos los links para cualquera de las  carpertas dado un rango de fechas, el nombre de la carpeta (tipo de imagen, en que banda se tomo), indicando apropiadamente la fecha de incio y la frecuencia con la que se toman, como se muestra a continuación estan generando para tres casos


In [12]:
# Metodo 1
hmiigr_urls = get_image_urls( image_type = 'hmiigr',start='01/01/2022 00:00:00', end= '01/01/2023', frequency='90T')

# Metodo 2
mdimag_urls =  get_image_urls( image_type = 'eit171', year= '1997')


Acá se estan probando el calculo de los parametros que se van implementando para 10 imagenes

In [5]:
import pandas as pd
from urllib.error import HTTPError

def compute_metrics(image_type:str, year:str = None, start:str = None, end:str = None,  frequency:str=  None ) -> pd.DataFrame:
    
    """
    Computes various image metrics for a set of images within a given date range and specified type and frequency.

    Parameters
    ----------
    start_date : str
        Start date for retrieving the images, formatted as 'YYYY-MM-DD'.
    
    end_date : str
        End date for retrieving the images, formatted as 'YYYY-MM-DD'.
    
    image_type : str
        Type/category of the image to retrieve. 
    
    frequency : str
        Frequency for retrieving images, e.g., 'daily', 'hourly'.

    Returns
    -------
    pd.DataFrame
        DataFrame containing computed metrics for each image, with one row per image 
        and columns for each feature. The index of the DataFrame corresponds to the datetime 
        extracted from the image URL.
        
    Notes
    -----
    The function fetches image URLs based on the provided date range, type, and frequency. 
    For each URL, it reads the image and computes a set of metrics, such as entropy, mean 
    intensity, and fractal dimension. If the image at a particular URL is inaccessible, it 
    skips that URL. The extracted date and time from the image URL is used as the index 
    for the resulting DataFrame.
    """

    image_urls = get_image_urls(image_type, year, start, end,  frequency )
    
    #Create an empty DataFrame to store the results
    df = pd.DataFrame(columns=['entropy', 'mean_intensity', 'std_deviation', 'fractal_dimension', 'skewness', 'kurtosis', 'uniformity', 'relative_smoothness', 
                                'taruma_contrast', 'taruma_directionality'])

    for url in image_urls[:30]:
        try:
            # Try to read the image
            image = io.imread(url)
            

        except HTTPError:
            #print(f"Unable to access image at {url}")
            continue
            
        # Compute the features
        entropy = sia.compute_entropy(image)
        mean_intensity = sia.compute_mean_intensity(image)
        std_deviation = sia.compute_standard_deviation(image)
        fractal_dimension = sia.compute_fractal_dimension(image)
        skewness = sia.compute_skewness(image)
        kurtosis = sia.compute_kurtosis(image)
        uniformity = sia.compute_uniformity(image)
        relative_smoothness = sia.compute_relative_smoothness(image)
        taruma_contrast = sia.compute_taruma_contrast(image)
        taruma_directionality = 0 #sia.compute_taruma_directionality(image)
        # Extract the date from the URL
        date_string = re.findall(r"\d{8}_\d{4}", url)[0]
        date = datetime.strptime(date_string, '%Y%m%d_%H%M')

        # Append the results to the DataFrame
        df.loc[date] = [entropy, mean_intensity, std_deviation, fractal_dimension, skewness, kurtosis, uniformity, relative_smoothness,taruma_contrast, taruma_directionality ]


    return df

# Este es el metodo que teniamos solo con una modificación en el orden de los parametros
#data = compute_metrics( image_type = 'hmiigr',start='01/01/2022 00:00:00', end= '01/01/2023', frequency='90T')
    
# Esta es la otra opción, para años anteriores a 2011
data = compute_metrics( image_type = 'mdimag', year= '1997')
#print(data)


In [6]:
data

,entropy,mean_intensity,std_deviation,fractal_dimension,skewness,kurtosis,uniformity,relative_smoothness,taruma_contrast,taruma_directionality
1997-01-01 00:03:00,5.417590,95.144212,57.504144,1.257417,-0.888002,2.185440,0.070904,0.999698,2719.653314,0.0
1997-01-01 01:39:00,5.419038,95.149872,57.511631,1.248718,-0.888353,2.183702,0.070875,0.999698,2720.902696,0.0
1997-01-01 03:15:00,5.417695,95.163834,57.523317,1.258937,-0.888992,2.182029,0.070924,0.999698,2722.530293,0.0
1997-01-01 04:51:00,5.413169,95.171887,57.528402,1.253572,-0.889578,2.182439,0.071039,0.999698,2722.883729,0.0
1997-01-01 06:27:00,5.414904,95.180019,57.533924,1.262469,-0.889164,2.182763,0.070984,0.999698,2723.305420,0.0
1997-01-01 08:03:00,5.418810,95.170457,57.532269,1.262711,-0.888616,2.182214,0.070841,0.999698,2723.319927,0.0
1997-01-01 09:36:00,5.991615,95.182611,60.820219,1.294799,-0.585863,2.038029,0.066542,0.999730,3095.945984,0.0
1997-01-01 11:15:00,5.420629,95.154806,57.558896,1.258725,-0.887872,2.176828,0.070962,0.999698,2727.525753,0.0
1997-01-01 12:51:00,5.415691,95.146852,57.546550,1.260586,-0.888315,2.178179,0.071100,0.999698,2725.933157,0.0
1997-01-01 14:27:00,5.412747,95.132169,57.528748,1.261612,-0.888884,2.180041,0.071080,0.999698,2723.664944,0.0
